In [2]:
import pandas as pd

### 파일 불러오기 감정지수, 토픽 비중

In [49]:
df_TopicCount=pd.read_excel(filename,usecols=['뉴스 기간','토픽 비중'])

In [50]:
df_TopicCount.fillna(0,inplace=True) #지수가 Nan인 날은 0으로 fill na

In [51]:
read_file=filename
df_SentimentalIndex=pd.read_excel(read_file,usecols=['뉴스 기간','감성지수'])

### 결합지수(0) 산출하기
#### : 감성지수 * 전체 토픽 80개 연관어 비중(%)
#### : col=['뉴스 기간','결합지수0]

In [52]:
LTerm=[]; LIndex0=[] #기간과 결합지수

In [53]:
LTerm=df_SentimentalIndex['뉴스 기간'].tolist()

In [55]:
for i in LTerm:
    Senti=df_SentimentalIndex.loc[df_SentimentalIndex['뉴스 기간']==i,['감성지수']].values[0][0]
    TopicPercent=df_TopicCount.loc[df_TopicCount['뉴스 기간']==i,['토픽 비중']].values[0][0]
    
    index=Senti*TopicPercent
    LIndex0.append(index)

In [57]:
data={'뉴스 기간':LTerm,'결합지수0':LIndex0}

In [61]:
df_index0=pd.DataFrame(data,columns=['뉴스 기간','결합지수0'])
df_index0.to_excel('df_index0.xlsx')

## 결합지수 1: 감성지수 * 토픽 비중 상위 20개 단어 비중 
### (1) 토픽 비중 연산 

In [ ]:
read_file=filename
dfTopicPercentage=pd.read_excel(read_file)

In [ ]:
read_file=filename
dfSentiment=pd.read_excel(read_file,usecols=['감정지수'])

In [ ]:
LTotal=[]; 
for i in range(262):
    LRow=[]
    
    TopicResult=dfTopicPercentage.iloc[i,3:83]
    Top20Topic=TopicResult.sort_values(ascending=[False])[:20]
    Top20Percent=sum(Top20Topic.tolist())*100
    
    Senti=dfSentiment.iloc[i].values[0]
    Index1=Senti*Top20Percent
    
    LRow.append(dfTopicPercentage.iloc[i,1]); LRow.append(Top20Topic.index.tolist())
    LRow.append(Top20Percent); LRow.append(Senti); LRow.append(Index1)
    
    LTotal.append(LRow)

In [ ]:
dfIndex1=pd.DataFrame(LTotal)
dfIndex1.to_excel(filename)

## 결합지수(2) 산출

### (1) 토픽 간 상관 관계 분석 

In [ ]:
df_TopicCount=pd.read_excel(filename) #전체 뉴스 기사에 대한 토픽비중(index(0) 간 상관계수)

In [ ]:
read_file=filename
df_SentimentalIndex=pd.read_excel(read_file,usecols=['뉴스 기간','감성지수'])

In [ ]:
LTotal=[]; LTerm=[]
for i in range(80): # 전체 기간에 대해서 
    ColTopic=df_TopicCount.iloc[:,i+3]  # 전체 기간의 토픽 비중 
    Senti=df_SentimentalIndex.iloc[:,1] #전체 기간의 감성지수 
    
    LCol=[Senti[i] * ColTopic[i] for i in range(262)] #결합지수 간 상관계수 산출 
    
    LTotal.append(LCol)
    
    LTerm.append(df_TopicCount.iloc[i,1])

In [ ]:
df_tmp=pd.DataFrame(LTotal).transpose()

In [ ]:
# 각 토픽들에 대해서 상관관계 구하기 
corr=[[df_tmp[i].corr(df_tmp[j]) for j in range(80)] for i in range(80)] 

In [ ]:
corr_total=[sum(r)for r in corr]
df_corr=pd.DataFrame(corr_total)

### (2) 감성지수 * 토픽 간 상관 상위 20개 토픽 단어 비중 (%)

In [ ]:
read_file=filename
df_SentimentalIndex=pd.read_excel(read_file,usecols=['뉴스 기간','감성지수'])

In [ ]:
read_file=filename
df_Topic=pd.read_excel(read_file)
df_Topic=df_Topic.iloc[:,3:]

In [ ]:
Top20Corr=[66,15,23,6,0,3,60,8,70,39,68,16,11,76,53,37,18,42,59,71]

In [ ]:
LIndex2=[]; LPercent=[]; LTerm=[]; LTopTopic=[]
for i in range(262):
    
    Senti=df_SentimentalIndex.iloc[i,1]
    TopicPercent=sum([df_Topic.iloc[i,j] for j in Top20Corr])*100 #퍼센트화
    
    Index2=Senti*TopicPercent
    LTerm.append(df_SentimentalIndex.iloc[i,0]); LIndex2.append(Index2); LPercent.append(TopicPercent)
    LTopTopic.append(Top20Corr)

In [ ]:
data={
    '뉴스 기간':LTerm, '상위 20 토픽':LTopTopic, '결합지수 2':LIndex2, '토픽 비중':LPercent
}

df_Index2=pd.DataFrame(data)
df_Index2.to_excel(filename)

## 결합지수(3) 산출

### (1) 토픽- 생산 간 상관계수 산출
### i토픽의 토픽-생산 상관계수=상관계수(i- 토픽 결합지수, 생산증가율)

In [ ]:
read_file=filename
df_TopicCount=pd.read_excel(read_file)

read_file=filename
df_SentimentalIndex=pd.read_excel(read_file,usecols=['뉴스 기간','감정지수'])

In [ ]:
read_file=filename
df_ProductionIndex=pd.read_excel(read_file,usecols=['수상 운송업'])

LProduction=df_ProductionIndex['수상 운송업'].tolist()
df_Production=pd.DataFrame(LProduction)

In [ ]:
LTotal=[]; LTerm=[]
for i in range(80): # 전체 기간에 대해서 
    ColTopic=df_TopicCount.iloc[:,i+3] # 전체 기간의 토픽 비중 
    Senti=df_SentimentalIndex.iloc[:,1] #전체 기간의 감성지수 
    
    LCol=[Senti[i] * ColTopic[i] for i in range(262)]
    
    LTotal.append(LCol) #토픽의 결합지수
    
    LTerm.append(df_TopicCount.iloc[i,1])
    
df_tmp=pd.DataFrame(LTotal).transpose()

In [ ]:
corr=[df_tmp[i].corr(df_Production[0]) for i in range(80) ]

In [ ]:
df_corr=pd.DataFrame(corr)
df_corr.to_excel(filename)

### (2) 결합지수(3) 산출
### 감성지수*토픽-생산 상관 상위 20개 토픽 단어 비중 (%)

In [ ]:
read_file=filename
df_SentimentalIndex=pd.read_excel(read_file,usecols=['뉴스 기간','감성지수'])

In [ ]:
read_file=filename
df_Topic=pd.read_excel(read_file)
df_Topic=df_Topic.iloc[:,3:]

In [ ]:
Top20Corr=[3,7,15,18,21,26,29,31,35,36,44,50,51,53,56,59,63,64,71,72] 

In [ ]:
LIndex3=[]; LPercent=[]; LTerm=[]; LTopTopic=[]
for i in range(262):
    Senti=df_SentimentalIndex.iloc[i,1]
    TopicPercent=sum([df_Topic.iloc[i,j] for j in Top20Corr])*100 #퍼센트화
    
    Index3=Senti*TopicPercent
    LTerm.append(df_SentimentalIndex.iloc[i,0]); LIndex3.append(Index3); LPercent.append(TopicPercent)
    LTopTopic.append(Top20Corr)

In [ ]:
data={
    '뉴스 기간':LTerm, '상위 20 토픽':LTopTopic, '결합지수 3':LIndex3, '토픽 비중':LPercent
}

df_Index2=pd.DataFrame(data)
df_Index2.to_excel(filename)